In [32]:
import pandas as pd 
import pickle
import sys 
sys.path.append("C:/Users/Danial Nasir Awan'/Desktop/Notion/Projects and portfolios/personal-endevours/projects/amazon-sales-predictor/libraries")
import training_xgb_models as trainer

In [33]:
skus_to_model_path =  "Data/currated/skus_to_be_modeled_2_10.pkl"
try:
    with open(f'{skus_to_model_path}', 'rb') as file:
        sku_to_model = pickle.load(file)

    print(f"Loaded sku len: {len(sku_to_model)}")
    
    print(f"Loaded skus: {sku_to_model}")
    print(type(sku_to_model))
except Exception as e:
    print(e)
    sku_to_model = []

Loaded sku len: 50
Loaded skus: ['PSDGLCRTWXC4', 'PSDC9FX9TH5V', 'PSDCF54V2TKG', 'PSDGFWMQCZCL', 'PSDQG6VWMMZV', 'PSDRZVCW8M36', 'PSD9TJN45ZFL', 'PSDQ8Z5JZLVM', 'PSDWKY269FGX', 'PSDZ66XTNWL8', 'PSDC8886WMZZ', 'PSDWLHH83Z5R', 'PSDZKF8NBH5Y', 'PSD8F689H89H', 'PSDHBJK5KMXH', 'PSDYGQ2FH553', 'PSDXZQRFV2GV', 'PSD8JNGGT2ZB', 'PSD9Y369LLY6', 'PSDR2J4XBVFC', 'PSDZZHZ6583N', 'PSDCKZ5HJTVT', 'PSD3CC3YVKR5', 'PSDGHY52544X', 'PSDWC2LWJJKJ', 'PSD9T5H3KJZ9', 'PSD9L59C3HX2', 'PSD5TGG3J4FZ', 'PSD6B28VJ233', 'PSDX35CJQQXT', 'PSDVY98HJBVN', 'PSD5TGG3JJVJ', 'PSDHRF54JQVX', 'PSDM29W2NH9Z', 'PSDFJ3TFV53J', 'PSDYT65JHBNK', 'PSDW2ZM8YFJ4', 'PSDCQ68TGM4N', 'PSDCX59NXJKT', 'PSD4FRY5C46Y', 'PSDRYF2HTVW4', 'PSDJYKCT3GBM', 'PSD3QK3YQCJ6', 'PSDTGCY8WBCL', 'PSDXY82RRJ2G', 'PSDL5FMCVWHQ', 'PSDFY9YGJ4F4', 'PSD25NRZYY2N', 'PSD35W2BN2XR', 'PSD9YBNBTH3N']
<class 'list'>


In [34]:
model_pickle_file = 'prophet_models_daily_2_10.pkl'

try:
    with open(f'data/models/{model_pickle_file}', 'rb') as file:
        models = pickle.load(file)

    print(f"Loaded sku len: {len(models)}")
    print(f"Loaded skus: {models}")
    print(type(models))
except:
    models= {}
s = []
for model_key, model in models.items():
    # Extract SKU from the model key (e.g., 'PSD6TL96YF9Z_mse' -> 'PSD6TL96YF9Z')
    sku = model_key.split('_')[0]
    s.append(sku)


Loaded sku len: 300
Loaded skus: {'PSDX35CJQQXT_custom': <prophet.forecaster.Prophet object at 0x000002637D07AF00>, 'PSDX35CJQQXT_mse': <prophet.forecaster.Prophet object at 0x000002630201A6F0>, 'PSDX35CJQQXT_huber_loss': <prophet.forecaster.Prophet object at 0x0000026302018D40>, 'PSDX35CJQQXT_poisson_loss': <prophet.forecaster.Prophet object at 0x0000026302021CA0>, 'PSDX35CJQQXT_mape': <prophet.forecaster.Prophet object at 0x0000026302022120>, 'PSDX35CJQQXT_remse': <prophet.forecaster.Prophet object at 0x0000026302023830>, 'PSD8JNGGT2ZB_custom': <prophet.forecaster.Prophet object at 0x0000026366E65490>, 'PSD8JNGGT2ZB_mse': <prophet.forecaster.Prophet object at 0x00000263020FFBC0>, 'PSD8JNGGT2ZB_huber_loss': <prophet.forecaster.Prophet object at 0x00000263020FC740>, 'PSD8JNGGT2ZB_poisson_loss': <prophet.forecaster.Prophet object at 0x00000263020FE630>, 'PSD8JNGGT2ZB_mape': <prophet.forecaster.Prophet object at 0x00000263020669C0>, 'PSD8JNGGT2ZB_remse': <prophet.forecaster.Prophet objec

In [36]:
try:
    prev_loss = pd.read_csv('data/currated/params_loss_daily_2_10.csv')
    prev_loss = prev_loss.drop(columns='Unnamed: 0')
except:
    prev_loss = pd.DataFrame(columns=['changepoint_prior_scale', 'changepoint_range',
       'daily_seasonality', 'growth', 'holidays_prior_scale', 'interval_width',
       'n_changepoints', 'seasonality_mode', 'seasonality_prior_scale',
       'uncertainty_samples', 'weekly_seasonality', 'yearly_seasonality',
       'sku', 'loss', 'loss_function','training_time'])
prev_loss.head(2)
prev_loss.shape

(11760, 16)

In [37]:
len(prev_loss['sku'].unique())

49

In [19]:
train_df, test_df,prod, current_skus = trainer.load_data(create_test_train_split = True, top_skus=False) 


C:\Users/Danial Nasir Awan'/Desktop/Notion/Projects and portfolios/personal-endevours/projects/amazon-sales-predictor/libraries\training_xgb_models.py:60: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  prod = cu.clean_prod(pd.read_csv('data/raw/prod.csv'))


In [38]:
train_df['purchase_date'].max()

Timestamp('2023-12-31 00:00:00')

In [39]:
test_df['purchase_date'].max()

Timestamp('2024-10-25 00:00:00')

In [ ]:
difference = list(set(sku_to_model) - set(s))
data_sku = difference
testing_data = test_df[test_df['sku'].isin(data_sku)]
training_data = train_df[train_df['sku'].isin(testing_data['sku'].unique())]
len(data_sku)


1

In [23]:
training_data['sku'].unique()

array(['PSD6B28VJ233'], dtype=object)

In [24]:
testing_data['sku'].unique()

array(['PSD6B28VJ233'], dtype=object)

In [25]:
loss_functions = [
    'custom',
    'mse',
    'huber_loss',
    'poisson_loss',
    'mape',
    'remse'
]
from scipy.stats import uniform
param_space = dict(
    growth=['linear','flat'],
    n_changepoints=range(5, 25, 5),
    changepoint_range=uniform(0.7, 0.2),
    yearly_seasonality=[True,False],
    weekly_seasonality=[True,False],
    daily_seasonality=[False,False],
    seasonality_mode=['additive', 'multiplicative'],
    seasonality_prior_scale=uniform(10.0, 20.0),
    changepoint_prior_scale=uniform(0.05, 0.2),
    interval_width=uniform(0.8, 0.2),
    uncertainty_samples=[1000, 2000],
    holidays_prior_scale=uniform(0.5, 2.0),
)
args_list = []
for sku in training_data['sku'].unique():
    for loss_name in loss_functions:
        # print(loss_name)
        args_list.append((sku, training_data[training_data['sku'] == sku],
                          testing_data[testing_data['sku'] == sku],
                          loss_name,param_space ,
                          models))

In [26]:
import logging
import prophet_modeling as pm 
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

results, models = pm.train(args_list)

Training Models:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Optimization terminated abnormally. Falling back to Newton.
Optimization terminated abnormally. Falling back to Newton.
Optimization terminated abnormally. Falling back to Newton.
Optimization terminated abnormally. Falling back to Newton.
Optimization terminated abnormally. Falling back to Newton.
Training Models:  17%|█▋        | 1/6 [03:13<16:05, 193.12s/it]

  0%|          | 0/30 [00:00<?, ?it/s]

Optimization terminated abnormally. Falling back to Newton.
Training Models:  33%|███▎      | 2/6 [06:28<12:58, 194.62s/it]

...

Training Models:  67%|██████▋   | 4/6 [12:53<06:25, 192.89s/it]

  0%|          | 0/30 [00:00<?, ?it/s]

Optimization terminated abnormally. Falling back to Newton.
Training Models:  83%|████████▎ | 5/6 [16:07<03:13, 193.25s/it]

  0%|          | 0/30 [00:00<?, ?it/s]

Training Models: 100%|██████████| 6/6 [19:14<00:00, 192.39s/it]


In [29]:
if (results):
    loss_results = pd.concat(results)
    complete_loss_df = pd.concat([loss_results,prev_loss])
    complete_loss_df.to_csv('data/currated/params_loss_daily_2_10.csv')

In [30]:
len(complete_loss_df['sku'].unique())

49

In [31]:

try:
    with open(f'data/models/{model_pickle_file}', 'wb') as file:
        pickle.dump(models,file)
except:
    print('not able to save models')